In [ ]:
import json
from os.path import join, isdir, isfile, abspath, dirname, splitext, basename, split
from collections import Counter

import pyperclip
from tqdm import tqdm
import nltk
from Levenshtein import distance
import pandas as pd

from derive_conceptualspace.create_spaces.translate_descriptions import get_langs
from derive_conceptualspace.util.mpl_tools import show_hist
from derive_conceptualspace.util.result_analysis_tools import df_to_latex, highlight_nonzero_max

In [ ]:
unique = lambda iterable: list({i:None for i in iterable}.keys())
flatten = lambda l: [item for sublist in l for item in sublist] 

BOOK_BASE = "/home/chris/Documents/UNI_neu/Masterarbeit/OTHER/study_behavior_analysis/src/"
path = "/home/chris/Documents/UNI_neu/Masterarbeit/OTHER/study_behavior_analysis/EducationalResource-2022-01-20.csv"

In [ ]:
def strip_title_description(origdf):
    df = origdf.copy()
    df["description"] = df["description"].str.strip().str.replace("\r\n", "\n") 
    df["description"] = df["description"].str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n")
    df["description"] = df["description"].str.replace("\n", " ") 
    df["description"] = df["description"].str.strip()
    df["title"] = df["title"].str.strip()
    return df    

def groupby_merge(df, by="title"):
    """INPUT: a dataset with multiple rows per title, each having their own description etc, 
       OUTPUT: a dataset with unique titles and lists of the original descriptions etc"""
    new_df = []
    for num, (title, grouped) in enumerate(df.groupby(by)):
        tmp = {}
        tmp[by] = title
        for col in set(df.columns)-{by}:
            tmp[col] = flatten((i if isinstance(i, (list, set, tuple)) else [i]) for i in grouped[col])
        new_df.append(tmp)
    return pd.DataFrame(new_df)

def create_cols(origdf1, origdf2):
    """this is done as preparation for concatenation to ensure no columns are dropped in the concat"""
    df1, df2 = origdf1.copy(), origdf2.copy()
    for column in set(df2.columns)-set(df1.columns):
        df1[column] = pd.NA
    for column in set(df1.columns)-set(df2.columns):
        df2[column] = pd.NA
    return df1, df2

def make_tuples(origdf, emptynone=False):
    """lists are not hashable and thus I cannot do stuff like `.unique()` on them """
    df = origdf.copy()
    for col in df.columns:
        if isinstance(df[col][0], (list, set)):
            if not emptynone:
                df[col] = df[col].apply(lambda x:tuple(x))
            else: 
                df[col] = df[col].apply(lambda x: tuple(x) if len(x) > 0 else pd.NA)
    return df

# filt_de = lambda df: df[df["detected_lang"] == "de"]
filt_len_single = lambda df: df[df["description"].str.count(" ") >= 80]
filt_len = lambda df, minwords=50: df[df["description"].apply(lambda x: any(i.count(" ") >= minwords for i in x if not pd.isna(i)))]
filt_de = lambda df: df[df["detected_lang"].apply(lambda x: "de" in [i for i in x if not pd.isna(i)])]

def squeeze_cols(origdf, keep_lists=False, subset=None):
    df = origdf.copy()
    for col in (subset or df.columns):
        if isinstance(df[col][0], (list, set, tuple)):
            tmp = df[col].apply(lambda x: unique(i for i in x if not pd.isna(i)) if x is not None else x)
            if keep_lists:
                tmp = tmp.apply(lambda x: [] if x is None or len(x) == 0 else x)
            else:
                tmp = tmp.apply(lambda x: pd.NA if x is None or len(x) == 0 else (x[0] if len(x) == 1 else x))
            df[col] = tmp
    return df

def stringify_listcols(origdf):
    df = origdf.copy()
    for col in df.columns:
        if isinstance(df[col][0], (list, set, tuple)):
            df[col] = df[col].apply(lambda x: str(x) if len(x) > 0 else "")
    return df

# First, get the "EducationalResource-2022-01-20.csv"

In [ ]:
df = pd.read_csv(path)
assert all(i.startswith('"') and i.endswith('"') and i[1:-1].isnumeric() for i in df["ddc_code"] if not pd.isna(i))

df = strip_title_description(df)
df["ddc_code"] = df["ddc_code"].str[1:-1]#.astype(pd.Int64Dtype())
df = df.drop(columns=["identifier", "contributor", "creator", "coverage", "date", "rights", "relation"])

display(df.describe())
print("\n\n")
display(df.head())
print("\n\n")
display(df["origin"].value_counts())

In [ ]:
print(df["type"].unique())
df["type"] = df["type"].str.replace("['SIP']", "SIP", regex=False).str.replace("['udemy', 'mooc']", "udemy_mooc", regex=False).str.replace("['OER']", "OER", regex=False)
print(df["format"].unique())
df["format"] = df["format"].str.replace("['CRS']", "CRS", regex=False)
df = df.set_index("id")
df = df.dropna(subset=["title"])
df.head()

In [ ]:
display(df["publisher"].value_counts())
seldom_publishers = [k for k,v in df["publisher"].value_counts().items() if v <= 20]
display(df[df["publisher"].isin(seldom_publishers)])

# Next, let's look at "course_dump_new.csv"

In [ ]:
cdn = pd.read_csv(join(BOOK_BASE, "data/course_data/db_dump_new", "course_dump_new.csv"))
cdn = cdn.drop(columns=["course_origin_id", "place", "start_time", "end_time", "end_semester", "date", "TF_IDF_scores",])
#display(cdn.head())
assert all(i.startswith('"\\"') and i.endswith('\\""') and i[3:-3].isnumeric() for i in cdn["ddc_code"] if not pd.isna(i))
cdn["ddc_code"] = cdn["ddc_code"].str[3:-3]
cdn = cdn.set_index("id")
cdn = strip_title_description(cdn).dropna(subset=["title"])
cdn = cdn.rename(columns={"origin_id":"origin"})

print(len(cdn))
cdn.head()

# And at "eduresource_dump.csv"

In [ ]:
edu = pd.read_csv(join(BOOK_BASE, "data/course_data/db_dump_new", "eduresource_dump.csv"))
print("#Entries for 'contributor' column:", len(edu[(~edu["contributor"].isna()) & (~edu["contributor"].isin([[], "[]"]))]))
print("#Entries for 'creator' column:", len(edu[(~edu["creator"].isna()) & (~edu["creator"].isin([[], "[]"]))]))
# edu.head()
edu = edu.drop(columns=["TF_IDF_scores", "contributor", "date", "relation", "rights", "origin_id"])
display(edu["format"].unique())
edu["format"] = edu["format"].str.replace('["udemy", "mooc"]', "udemy_mooc", regex=False).str.replace('["CRS"]', "CRS", regex=False).str.replace('"video/mp4"', "mp4", regex=False).str.replace('[]', "", regex=False)
display(edu["type"].unique())
edu["type"] = edu["type"].str.replace('["SIP"]', "SIP", regex=False).str.replace('["udemy", "mooc"]', "udemy_mooc", regex=False).str.replace('["video", "OER"]', "video_OER", regex=False).str.replace('["WEB"]', "web", regex=False)
display(edu["type"].unique())
edu = edu.set_index("id")
assert all(i.startswith('"\\"') and i.endswith('\\""') and i[3:-3].isnumeric() for i in edu["ddc_code"] if not pd.isna(i))
edu["ddc_code"] = edu["ddc_code"].str[3:-3]
edu = strip_title_description(edu).dropna(subset=["title"])
print(len(edu))
edu.head()

## Merging them

In [ ]:
print(len(df))
print(len(cdn))
print(len(edu))
display(df.head(2))
display(cdn.head(2))
display(edu.head(2))
print(list(df.dtypes))
print(list(cdn.dtypes))
print(list(edu.dtypes))

### merging. First the dumps `df` and `cdn`

In [ ]:
cdn["type"] = "SIP"
cdn["dset_origin"] = "course_dump_new"
df["dset_origin"] = "EducationalResource-2022-01-20"
edu["dset_origin"] = "eduresource_dump"

In [ ]:
print(len(set(df["title"])), len(set(cdn["title"])), len(set(df["title"])&set(cdn["title"])), len(set(df["title"])|set(cdn["title"])))

In [ ]:
df, cdn = create_cols(stringify_listcols(df), stringify_listcols(cdn))
df_cdn = groupby_merge(pd.concat([df, cdn]))
df_cdn.head()
#df_cdn[df_cdn["description"].apply(lambda x: len(x) > 1)]

#### merging with edu

In [ ]:
df_cdn, edu = create_cols(df_cdn, stringify_listcols(edu))
df_cdn_edu = groupby_merge(pd.concat([df_cdn, edu]))
df_cdn_edu.head()

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', 25, 'display.expand_frame_repr', False, 'display.max_colwidth', 20, 'display.float_format', '{:.4f}'.format):
    display(df_cdn_edu)

# Merging with the old one

## Preparing the old one...

In [ ]:
orig = pd.read_csv("/home/chris/Documents/UNI_neu/Masterarbeit/data_new/siddata2021/raw_descriptions.csv")
orig["Name"] = orig["Name"].str.strip()
orig["Beschreibung"] = orig["Beschreibung"].str.strip()
orig = orig.rename(columns=dict(VeranstaltungsNummer="veranstaltungsnummer", Name="title", Untertitel="subtitle", Beschreibung="description"))
orig["description"] = orig["description"].str.strip().str.replace("\r\n", "\n")
orig["description"] = orig["description"].str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n").str.replace("\n\n", "\n")
orig["description"] = orig["description"].str.replace("\n", " ")
orig["description"] = orig["description"].str.strip()
orig["description"] = orig["description"].str.replace(r'<.*?>', '', regex=True)
orig["dset_origin"] = "old_dump"
orig = strip_title_description(orig)
orig

In [ ]:
df_cdn_edu, orig = create_cols(df_cdn_edu, orig)
alls = groupby_merge(pd.concat([df_cdn_edu, orig]))
alls.head()

In [ ]:
alls = alls[alls["description"].apply(lambda x: not all(pd.isna(i) for i in x))] #filter those that have ANY existing description
alls = alls.reset_index().drop(columns="index")

In [ ]:
alls = squeeze_cols(alls, keep_lists=True)
#alls_simple = alls[alls["description"].apply(lambda x: len(x) < 2)].reset_index().drop(columns="index")
#alls_multi = alls[~alls["description"].apply(lambda x: len(x) < 2)].reset_index().drop(columns="index")

In [ ]:
alls.to_csv("/home/chris/Documents/UNI_neu/Masterarbeit/data_new/siddata2022_again.csv")

So, we DID SQUEEZE the description. No matter where the course is from, we benefit from more descriptions.
So if UOS has a different "Analysis 1" Course with a different description than Uni Bremen, good for us, more words in the description for any course of the name "Analysis 1".

HOWEVER, for everything else, that is not the case and we cannot squeeze! Best example: For the veranstaltungsnummer it is relevant if the course was at UOS or at Bremen, so we must be able to recover which veranstaltungsnummer belonged to which origin, and the easiest for now is to just keep the lists of the same length.  
EDIT: this is BS! As shown below, veranstaltungsnummer ONLY EXISTS for the original UOS export!!

# It's saved. We're done. The rest here is just analysis.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import plotly.express as px

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
alls50 = filt_len(alls, minwords=20).reset_index().drop(columns="index")
#alls50["creator"].value_counts() is completely empty -> wFe can drop it
#any(i > 1 for i in alls50["identifier"][alls50["identifier"].str.len() > 2].value_counts().values) -> identifier is unique, we can drop it
alls50 = alls50.drop(columns=["creator", "identifier"])
alls50 = make_tuples(alls50, emptynone=False)
alls50.head()

In [ ]:
alls50.loc[:, "source"] = None
alls50.loc[alls50["dset_origin"].apply(lambda x: "old_dump" in x), "source"] = "2021 Dump"
alls50.loc[alls50["dset_origin"].apply(lambda x: not "old_dump" in x and "course_dump_new" in x), "source"] = "2022 Dump"
alls50.loc[alls50["dset_origin"].apply(lambda x: not "old_dump" in x and not "course_dump_new" in x), "source"] = "Educational-Resources"

with PdfPages("/home/chris/Documents/UNI_neu/Masterarbeit/MastersThesisText/graphics/dataset_new/course_source_df.pdf") as pdf:
    fig1, ax1 = plt.subplots()
    ax1.pie(alls50["source"].value_counts().values, labels=alls50["source"].value_counts().index, autopct=lambda p: '{:.0f}'.format(p * len(alls50) / 100))
    ax1.axis('equal')
    plt.show()
    pdf.savefig(fig1, bbox_inches='tight')

alls50 = alls50.drop(columns="dset_origin")

In [ ]:
assert len(alls50[(alls50["source"] != "2021 Dump") & (alls50["veranstaltungsnummer"].apply(lambda x: len(x) > 1))]) == 0 
#veranstaltungsnummer ONLY EXISTS for the original UOS export

In [ ]:
alls50["type"] = alls50["type"].apply(lambda x: x[0] if not pd.isna(x) and len(x) > 0 else pd.NA)
#there is one single entity that has two types, we'll just ignore that.
alls50.loc[alls50["source"] == "2021 Dump", "type"] = "SIP"

tmp = alls50[["source", "type"]].reset_index().drop(columns="index")
tmp["type"] = tmp["type"].fillna("unknown")
count_df = pd.DataFrame(tmp.value_counts(dropna=False)).rename(columns={0:"Count"})
count_df

In [ ]:
fig = px.sunburst(count_df.reset_index(), path=['source', 'type'], values="Count")
fig.update_traces(textinfo="label+value") # Any combination of [‘label’, ‘text’, ‘value’, ‘current path’, ‘percent root’, ‘percent entry’, ‘percent parent’] 
fig.show()

In [ ]:
trnsl = {"f6f8ced1-2a20-45de-ad27-46297617f247": "de.uni-hannover.studip", "8f83372d-7e8a-4720-bd63-90572e8a3d26": "de.uni-osnabrueck.studip", "1cf496ff-6e14-4c3b-89fc-304c974cfe8c": "de.uni-bremen.elearning"}
tmp = alls50[["publisher", "origin"]]
tmp = tmp.dropna()
tmp["origin"] = tmp["origin"].apply(lambda x: tuple(trnsl.get(i, i) for i in x if not pd.isna(i)))
tmp["default_publisher"] = tmp["publisher"].apply(lambda x: any(i == "default" for i in x if not pd.isna(i)))
tmp["publisher"] = tmp["publisher"].apply(lambda x: tuple(i for i in x if not pd.isna(i) and i != "default"))

tmp = tmp[tmp["publisher"] != tmp["origin"]]
#print("So many ones have a differing origin and publisher:", len(tmp))
#display(tmp.head())

overwrite_publisher = {n: sorted(list(set(i["publisher"])|set(i["origin"])|({"default"} if i["default_publisher"] else set()))) for n,i in tmp.iterrows()}
for ind, publisher in overwrite_publisher.items():
    alls50.loc[ind, "publisher"] = publisher
    

#those of the 2021 dump don't mention it, but their publisher is osnabrueck.
alls50.loc[alls50["source"] == "2021 Dump", "publisher"] = alls50.loc[alls50["source"] == "2021 Dump", "publisher"].fillna("de.uni-osnabrueck.studip").apply(lambda x: x if "de.uni-osnabrueck.studip" in x else tuple(sorted(list(x)+["de.uni-osnabrueck.studip"])))
alls50["publisher"] = alls50["publisher"].apply(lambda x: tuple([x]) if isinstance(x, str) else x)
alls50 = alls50.drop(columns="origin")

In [ ]:
for val in trnsl.values():
    alls50[f"is_{val}"] = alls50["publisher"].apply(lambda x: val in x)
alls50["is_other"] = alls50["publisher"].apply(lambda x: len(set(i for i in x if i not in trnsl.values()))>0)
alls50 = alls50.rename(columns={"is_de.uni-hannover.studip": "is_hannover", "is_de.uni-osnabrueck.studip": "is_uos", "is_de.uni-bremen.elearning": "is_bremen"}).drop(columns="publisher")

In [ ]:
alls50["format"] = alls50["format"].apply(lambda x: x if x != ("CRS", "udemy_mooc") else ("CRS+mooc",))
assert not any(alls50["format"].apply(lambda x: len(x) > 1))  #there is no course with >1 format
alls50["format"] = alls50["format"].apply(lambda x: pd.NA if len(x) == 0 else x[0])
alls50.loc[alls50["format"] == "", "format"] = pd.NA
alls50.loc[alls50["type"] == "SIP", "format"] = "CRS"

In [ ]:
subject_temp = alls50["subject"].apply(lambda x: [i for i in x if not (pd.isna(i) or i in [[],"[]"])])
alls50["subject"] = subject_temp.apply(lambda x: pd.NA if len(x) == 0 else flatten([eval(i) for i in x]))
#alls50["subject"].dropna()[228] # PERFECT KEYWORD-CANDIDATES!!

In [ ]:
alls50.loc[alls50["coverage"].apply(lambda x: len(x) > 1),"coverage"] = alls50[alls50["coverage"].apply(lambda x: len(x) > 1)]["coverage"].apply(lambda x: (",".join(x),))
#sind bei len>=20 nur 3 stück, die appenden wir einfach, fertig.

assert list(dict(alls50["coverage"].apply(lambda x: len(x)).value_counts()).keys()) == [0, 1] #there is no course with > coverage
alls50["coverage"] = alls50["coverage"].apply(lambda x: pd.NA if len(x) == 0 else x[0]).value_counts()

In [ ]:
#only languages I see are de & en
lans = alls50["language"].apply(lambda x: unique([j.replace("Deutsch","de").replace("English","en") for j in flatten([i.split(",") for i in x])]))
alls50["language"] = lans.apply(lambda x: pd.NA if len(x) == 0 else x)

In [ ]:
assert len([i for i in alls50["coverage"].unique() if not pd.isna(i)]) == 0
alls50 = alls50.drop(columns="coverage")

In [ ]:
for col in ["ddc_code", "start_semester", "url"]:
    alls50[col] = alls50[col].apply(lambda x: pd.NA if len(x) == 0 else x)

In [ ]:
len(alls50[alls50["type"].isna()]) <= 10
alls50 = alls50[~alls50["type"].isna()]

In [ ]:
alls50.head()

Ok, das reicht uns erstmal.
* **title, description, language** sind klar.
* **format, type, source**  sind meta-infos, die aber nur für wenige gegeben sind.
* **subject** sind lists of keywords for the course. PERFECT to add to the description AND to automatically take as keyword-candidates!!
* **subtitle** kann man optional zu den descriptions adden und dann behandeln wie eine descriptions.
* **ddc_code, veransltaltungsnummer**  sind possible targets (!)
* **is_hannover, is_bremen, ...** sind auch possible targets
* **start_semester, url** sind for now.

## Okay, I'll translate them now.

In [ ]:
unique_descs = alls50["description"].explode().dropna().unique()

In [ ]:
lans = get_langs(unique_descs, assert_len=False)

In [ ]:
arg = alls50[alls50["description"].apply(lambda x:len(set(lans[i] for i in x)) > 1)]
arg = pd.DataFrame(arg[["title", "description"]].explode("description"))
arg["lang"] = arg["description"].apply(lambda x: lans[x])
with pd.option_context('display.max_rows', 5, 'display.max_colwidth', 4000):
    display(arg)

In [ ]:
alls50["detected_lang"] = alls50["description"].apply(lambda x:set(lans[i] for i in x))
display(alls50["detected_lang"].apply(lambda x: "de" if "de" in x else list(x)[0]).value_counts()[:5])
alls50["detected_lang"] = alls50["detected_lang"].apply(lambda x: "de" if "de" in x else ("en" if "en" in x else "other"))

In [ ]:
tmp

In [ ]:
tmp = alls50.copy()
tmp["format"] = tmp["format"].fillna("unknown")
tmp[tmp["format"] == "CRS+mooc"] = "udemy_mooc"
tmp["type"] = tmp["type"].fillna("unknown")
tmp["is_uos"] = tmp["is_uos"].apply(lambda x: "UOS" if x else "other")
count_df = pd.DataFrame({f"Count≥{x}": filt_len(tmp, x)[["source", "type", "format", "is_uos", "detected_lang"]].value_counts(dropna=False) for x in [20, 50, 200, 500]}).sort_index()
count_df.style.format(precision=0, na_rep="-")

In [ ]:
count_df.index.names = [i.capitalize() for i in count_df.index.names]
styles = [{'selector': 'th', 'props': [('vertical-align','top')]}]  #('border-style', 'solid')  #see https://stackoverflow.com/a/55904239/5122790
styler = lambda df: df.style.format(precision=0, na_rep="-").set_table_styles(styles)
latex = df_to_latex(count_df, styler, rotate=None)
latex = latex.replace("Is_uos", "Uni").replace("Detected_lang", "Language").replace("udemy_mooc", "Udemy-\\acrshort{mooc}").replace("Count", "\#Words").replace("UOS","\\acrshort{uos}")
#latex = latex.replace("\\textbf{Educational-Resources}", "\\specialcell[l]{\\textbf{Educational\\\\Resources}}")

pyperclip.copy(latex)
#caption: "\caption[Metadata of the SIDDATA-Dataset]{Metadata of the SIDDATA-Dataset. Languages are reported as detected (see \ref{ap:translating}), other metadata as it was available in the dump. The individual columns is the number of entities whose description has at least 20, 50, 200 or 500 words.}"

In [ ]:
for i in range(len(count_df.index.names)):
    display(count_df.sum(axis=0, level=i).style.format(precision=0, na_rep="-"))

In [ ]:
fig = px.sunburst(count_df.reset_index(), path=count_df.index.names, values="Count≥50")
fig.update_traces(textinfo="label+value") # Any combination of [‘label’, ‘text’, ‘value’, ‘current path’, ‘percent root’, ‘percent entry’, ‘percent parent’] 
fig.show()

# Saving

In [ ]:
alls50.to_csv("/home/chris/Documents/UNI_neu/Masterarbeit/data_new/siddata2022_again.csv")